#Creating Node Class

In [352]:
class Node:
        
    def __init__(self, name, parent, g, h, f, transport):                                          # Initializing the class
        self.name = name
        self.parent = parent
        self.g = g                                                                      # Distance to start node
        self.h = h                                                                      # Distance to goal node
        self.f = f      
        self.transport = transport                                                                 # Total cost
            
    def __eq__(self, other):                                                            # Comparing two nodes
        return self.name == other.name
    
    def __lt__(self, other):                                                            # Sorting nodes
        return self.f < other.f
    
    def __repr__(self):                                                                 # Printing nodes
        return ('({0},{1})'.format(self.name, self.f))
    
    def printNode(self):                                                                # Customized Printing of nodes
        print(self.name, end = " - ")
        print(self.parent, end = " : ")
        print(self.g, end = " : ")
        print(self.h, end=" : ")
        print(self.f)

#Creating Graph Class

In [353]:
class Graph:
    
    def __init__(self, graph_dict=None, directed=True):                                 # Initialize the class
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
                
    def make_undirected(self):                                                          # Create an undirected graph by adding symmetric edges
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
                    
    def connect(self, A, B, time=1, cost = 1 ):                                                # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
        self.graph_dict.setdefault(A, {})[B] = [time, cost]
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = [time, cost]
               
          

    def getNode(self, city, heuristics, end,means_of_transport ,add=0):   
                                         # Get a specific neighbour which has minimum cost
        nodes = []
        if city  not in self.graph_dict:
          return nodes
        for (b,[time, cost]) in self.graph_dict[city].items():
            nodes.append(Node(city, b, time+cost+add, heuristics[b+'-'+end][0]+heuristics[b+'-'+end][1], time+cost+heuristics[b+'-'+end][0]+heuristics[b+'-'+end][1]+add, means_of_transport ))
        return nodes
   
        
    def printgraph(self):                                                               # Function to print each edge in the entire graph
         for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                print (self.graph_dict.setdefault(a,{})[b], end = " : ")
                print(a, end = " - ")
                print(b)


In [354]:

def sort_heuristic(frontier):
  frontier = (sorted(frontier, key=lambda x: x.f))
  return frontier


In [355]:

def backtrack(open_list, start, end):
  map = {}
  path = list()
  trans = list()
  for item in open_list:
    map[item.parent] = [item.name, item.transport]
  val = end
  while(True):
    path.append(val)
    if(val == start):
      break
    trans.append(map[val][1])
    val = map[val][0]
    
  path.reverse()
  trans.reverse()
 
  return path, trans







#Defining A_Star function

In [356]:

def A_Star(graph1, graph2, heuristics1,heuristics2, start, end):


  if((end not in graph1.graph_dict) and (end not in graph2.graph_dict)):                                                    # Incase the goal state does not exist
    print("\n\n---------------------------\nGOAL STATE DOES NOT EXIST\n---------------------------\n\n")
    return  None

  open_list = list()
  path = list()     
  trans = list()        
                                            # Will store the path we are taking
  curr_node1 = graph1.getNode(start,heuristics1, end, 'road')  
  curr_node2 = graph2.getNode(start, heuristics2, end, 'flight')

  frontier = list()
  frontier = curr_node1 + curr_node2
  frontier = sort_heuristic(frontier)


  curr_node = frontier[0]
  visited = []
  
  visited.append(curr_node.name)

  while(True):                                                     # Runs Until we cannot find the goal state or
     

      open_list.append(curr_node)

     # path.append(curr_node.parent)
      if(curr_node.parent==end):
        print("Total cost: ", curr_node.f)
        path, trans = backtrack(open_list, start, end)
        break

      visited.append(curr_node.parent)
      

      if(len(frontier)>0):
        frontier.pop(0)

      curr_node1 =  graph1.getNode(curr_node.parent,heuristics1, end, 'road',curr_node.g)  
      curr_node2 = graph2.getNode(curr_node.parent, heuristics2, end, 'flight',curr_node.g)

      frontier.extend(curr_node1)
      frontier.extend(curr_node2)

     
      frontier = sort_heuristic(frontier)
 

      while(len(frontier)> 0 and frontier[0].parent in visited):
        frontier.pop(0) 

      if(len(frontier)): 
        curr_node  = frontier[0]
      else:
        break


  return path, trans

###Creating graph for given city with time and cost for Bus tranportation

---



In [357]:

# Create a graph  -- by road
graph1 = Graph()

# Create graph connections (Actual distance)
                                   #time #cost
graph1.connect('Ahmedabad', 'surat', 70, 50)
graph1.connect('vadodara', 'surat',30,30)
# graph1.connect('Rajkot', 'surat',30,30)
graph1.connect('vadodara', 'patna',10,10)
graph1.connect('Ahmedabad', 'vadodara', 40,40)
graph1.connect('vadodara', 'Rajkot', 100,60)
graph1. make_undirected()
    


###Creating graph for given city with time and cost for flight tranportation


In [358]:
# Create a graph -- by flight
graph2 = Graph()
    
# Create graph connections (Actual distance)
                                   #time #cost
graph2.connect('Ahmedabad', 'surat',10 , 500)
graph2.connect('Ahmedabad', 'Rajkot', 15,400)
graph2. make_undirected()
    


##Heuristic Data1 (Bus Travel distance)

In [359]:
# Make graph undirected, create symmetric connections
# graph.make_undirected()
    
# Create heuristics (straight-line distance, air-travel distance) for Destination Navsari
heuristics1 = {}
heuristics1['Ahmedabad-surat'] = [1, 1 ]
heuristics1['Ahmedabad-Ahmedabad'] = [0,0 ]
heuristics1['Ahmedabad-vadodara'] = [1, 1 ]
heuristics1['Ahmedabad-Rajkot'] = [1, 1 ]
heuristics1['Ahmedabad-patna'] = [1, 1 ]
heuristics1['Rajkot-Rajkot'] = [0, 0 ]
heuristics1['Rajkot-Ahmedabad'] = [1, 1 ]
heuristics1['Rajkot-vadodara'] = [1, 1 ]
heuristics1['Rajkot-surat'] = [1, 1 ]
heuristics1['Rajkot-patna'] = [1, 1 ]

heuristics1['vadodara-surat'] = [1, 1 ]
heuristics1['vadodara-Ahmedabad'] = [1, 1 ]
heuristics1['vadodara-Rajkot'] = [1, 1 ]
heuristics1['vadodara-vadodara'] = [0, 0 ]
heuristics1['vadodara-patna'] = [0, 0 ]
heuristics1['surat-vadodara'] = [1, 1 ]
heuristics1['surat-Rajkot'] = [1, 1]
heuristics1['surat-Ahmedabad'] = [1, 1 ]
heuristics1['surat-surat'] = [0, 0 ]
heuristics1['surat-patna'] = [0, 0 ]


heuristics1['patna-patna'] = [0, 0 ]
heuristics1['patna-Ahmedabad'] = [0, 0 ]
heuristics1['patna-vadodara'] = [0, 0 ]
heuristics1['patna-Rajkot'] = [0, 0 ]
heuristics1['patna-surat'] = [0, 0 ]
# print(heuristics1)

##Heuristic Data2 (Flight Travel distance)

In [360]:
# Create heuristics (distance, train distance) for Destination Navsari
heuristics2= {}

heuristics2['Ahmedabad-surat'] = [1, 1]
heuristics2['Ahmedabad-Ahmedabad'] = [0,0 ]
heuristics2['Ahmedabad-vadodara'] = [1, 1 ]
heuristics2['Ahmedabad-Rajkot'] = [1, 1 ]
heuristics2['Rajkot-Rajkot'] = [0, 0 ]
heuristics2['Rajkot-Ahmedabad'] = [1, 1 ]
heuristics2['Rajkot-vadodara'] = [1, 1]
heuristics2['Rajkot-surat'] = [1, 1 ]
heuristics2['vadodara-surat'] = [1, 1 ]
heuristics2['vadodara-Ahmedabad'] = [1, 1 ]
heuristics2['vadodara-Rajkot'] = [1, 1]
heuristics2['vadodara-vadodara'] = [0, 0 ]
heuristics2['surat-vadodara'] = [1, 1]
heuristics2['surat-Rajkot'] = [1, 1 ]
heuristics2['surat-Ahmedabad'] = [1, 1]
heuristics2['surat-surat'] = [0, 0 ]
heuristics2['Rajkot-patna'] = [1, 1 ]
heuristics2['vadodara-patna'] = [0, 0 ]
heuristics2['surat-patna'] = [0, 0 ]
heuristics2['Ahmedabad-patna'] = [1, 1 ]
heuristics2['patna-patna'] = [0, 0 ]
heuristics2['patna-Ahmedabad'] = [0, 0 ]
heuristics2['patna-vadodara'] = [0, 0 ]
heuristics2['patna-Rajkot'] = [0, 0 ]
heuristics2['patna-surat'] = [0, 0 ]

###Calculating the throught both air and train distances

In [361]:


# Run search algorithm

start = 'Ahmedabad'
Goal  = 'Rajkot'

path, trans = A_Star(graph1, graph2 ,heuristics1,heuristics2 ,start,Goal)        


for index in range(len(path)-1):
  print(path[index], end='') 
  print(' ---', trans[index], '---> ', end='')

print(path[len(path)-1])

    


Total cost:  240
Ahmedabad --- road ---> vadodara --- road ---> Rajkot
